In [ ]:
from pathlib import Path
dataset_path = Path('../input/petfinder-pawpularity-score/')

In [ ]:
N_FOLDS = 10
FOLDS = range(N_FOLDS)
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE
GRADIENT_ACCUMULATION = 1
IMAGE_SIZE = 384
BACKBONE = 'swin_large_patch4_window12_384'
MAX_LR = 2e-5
DECAY = 0.1
EPOCHS = 5
WORKERS = 2
VERBOSE = None
MIXUP = 0.3
NUM_TARGET = 50

IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
import json
from tqdm import tqdm
import random
import time
import gc

from scipy.special import expit, logit, softmax
from sklearn.model_selection import KFold
from skimage.transform import rescale, resize, downscale_local_mean
import cv2
import albumentations

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler



In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
def seed_torch(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
    if torch.backends.cudnn.is_available:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [ ]:
valid_transforms = albumentations.Compose([
    albumentations.CenterCrop(IMAGE_SIZE, IMAGE_SIZE, always_apply=True, p=1),
    albumentations.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=1., always_apply=True, p=1),
])


In [ ]:
def load_image(path):
    #path = f'../input/petfinder-pawpularity-score/train/{id}.jpg'
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

class PetDataset(Dataset):
    def __init__(self,
                 data,
                 image_size,
                 target=True,
                 aug=None,
                 channel_first=True,
                ):
        super(PetDataset, self).__init__()
        self.data = data
        self.image_size = image_size
        self.target = target
        self.aug = aug
        self.channel_first = channel_first
        
    def __len__(self):
        return len(self.data) 
    
    def __getitem__(self, idx):
        image = load_image(self.data.path[idx])
        image = image.astype('float32')
        image /= 255.
        h, w, c = image.shape
        if h > w:
            image = resize(image, (int(round(self.image_size * h / w)), self.image_size), preserve_range=True, anti_aliasing=True)
        else:
            image = resize(image, (self.image_size, int(round(self.image_size * w / h))), preserve_range=True, anti_aliasing=True)
        if self.aug is not None:
            this = self.aug(image=image)
            image = this['image']
        if self.channel_first:
            image = image.transpose(2, 0, 1)
        target = 0
        binary_target = 0
        if self.target:
            target = self.data.norm_score[idx]
            binary_target = int(round(target * NUM_TARGET))
        return {'image':torch.tensor(image, dtype=torch.half), 
                'target':torch.tensor([target], dtype=torch.half),
                'binary_target':torch.tensor([binary_target], dtype=torch.long),
               }

In [ ]:
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df


In [ ]:
dataset = PetDataset(test_df, IMAGE_SIZE, aug=valid_transforms, target=False, channel_first=False)
#plt.imshow(dataset.__getitem__(1)['image'].data)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

class PetModel(nn.Module):
    def __init__(self, backbone_name, pretrained=True, loss=False):
        super(PetModel, self).__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=NUM_TARGET, drop_rate=0.00, drop_path_rate=0.00)
        self.loss = loss
        
    def forward(self, input_dict, mixup=0):
        x = input_dict['image']
        if mixup:
            batch_size = x.size(0)
            perm = torch.randperm(batch_size).to(x.device)
            x_perm = x[perm]
            beta = torch.distributions.beta.Beta(torch.tensor([0.5]), torch.tensor([0.5]))
            alpha = beta.sample(torch.tensor([batch_size])).to(x.device)
            alpha = alpha.view((alpha.shape[0], 1, 1, 1))
            x = alpha*x + (1 - alpha)*x_perm
                        
        x = self.backbone(x)
        pred = torch.sigmoid(x).sum(1) / NUM_TARGET
        
        loss = None
        if self.loss:
            #target = input_dict['target']
            binary_target = input_dict['binary_target']
            #binary_target = 1 - np.eye(num_class)[target].cumsum(1)[:, :-1]
            binary_target = 1. - F.one_hot(binary_target, NUM_TARGET+1).cumsum(-1)[:, :, :-1].transpose(1, 2).squeeze(-1)
            #print(binary_target.shape, x.shape)
            if mixup:
                target_perm = binary_target[perm]
                alpha = alpha.view((alpha.shape[0], 1))
                target = alpha*binary_target + (1 - alpha)*target_perm
            loss = criterion(x, binary_target)
        return {
            'pred' : pred,
            'loss' : loss,            
        }

In [ ]:
def test_epoch(loader, model, device):

    model.eval()
    LOGITS = []
    
    with torch.no_grad():
        if 1:
            if VERBOSE is not None:
                bar = tqdm(range(len(loader)))
            else:
                bar = range(len(loader))
            load_iter = iter(loader)
            batch = load_iter.next()
            batch = {k:batch[k].to(device, non_blocking=True) for k in batch.keys() }

            for i in bar:
                input_dict = batch.copy()
                if i + 1 < len(loader):
                    batch = load_iter.next()
                    batch = {k:batch[k].to(device, non_blocking=True) for k in batch.keys() }
                    
                out_dict = model(input_dict)
                pred = out_dict['pred']
                LOGITS.append(pred.detach())
                
    
    LOGITS = torch.cat(LOGITS).cpu().numpy().ravel()
    
    return LOGITS

def load_checkpoint(fold, seed, dirname, fname, test=False):
    model = PetModel(BACKBONE, pretrained=(not test), loss=True).to(device)
    checkpoint = torch.load('../input/%s/%s_%d_%d.pt' % (dirname, fname, fold, seed))
    model.load_state_dict(checkpoint['model'])
    model = model.half()
    return model

VERBOSE = None

In [ ]:
preds = []
    

In [ ]:
fname = 'pet_052'
print(fname)
#FOLDS = [1,2,3,4]
device = torch.device('cuda')
test_dataset = PetDataset(test_df, IMAGE_SIZE, target=False, aug=valid_transforms)
for seed in [0]:
    for fold in tqdm(FOLDS):
        seed_torch(seed)
                        

        test_data_loader = DataLoader(
            test_dataset,
            batch_size=VALID_BATCH_SIZE,
            num_workers=WORKERS,
            shuffle=False,
            #pin_memory=True,
        )
        model = load_checkpoint(fold, seed, 'pet-models',  fname, test=True)
        pred = test_epoch(test_data_loader, model, device) * 100
        preds.append(pred)
        del test_data_loader, model
        gc.collect()

In [ ]:
fname = 'pet_064'
print(fname)
#FOLDS = [1,2,3,4]
device = torch.device('cuda')
for seed in [0]:
    for fold in tqdm(FOLDS):
        seed_torch(seed)
                        
        test_data_loader = DataLoader(
            test_dataset,
            batch_size=VALID_BATCH_SIZE,
            num_workers=WORKERS,
            shuffle=False,
            #pin_memory=True,
        )
        model = load_checkpoint(fold, seed, 'pet64model',  fname, test=True)
        pred = test_epoch(test_data_loader, model, device) * 100
        preds.append(pred)
        del test_data_loader, model
        gc.collect()

In [ ]:
preds = np.mean(preds, axis=0)
sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
sample_df['Pawpularity'] = preds
sample_df.to_csv('submission.csv',index=False)


In [ ]:
sample_df